In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [2]:
from diffusion.data_loaders.spinkick_dataset import SpinkickMotionDataset
dataset = SpinkickMotionDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_spinkick.txt")
len(dataset)

1560

In [4]:
x = dataset[0]

x.shape

torch.Size([78, 35])

### Model NN

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # not used in the final model
        x = x + self.pe[:x.shape[0], :]
        return self.dropout(x)


class TimestepEmbedder(nn.Module):
    def __init__(self, latent_dim, sequence_pos_encoder):
        super().__init__()
        self.latent_dim = latent_dim
        self.sequence_pos_encoder = sequence_pos_encoder

        time_embed_dim = self.latent_dim
        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

    def forward(self, timesteps):
        return self.time_embed(self.sequence_pos_encoder.pe[timesteps])

In [15]:

class MotionTransformer(nn.Module):
    def __init__(self, pos_dim, vel_dim, latent_dim=256, ff_size=1024, num_layers=8, num_heads=4, dropout=0.1, activation="gelu"):
        super(MotionTransformer, self).__init__()
        
        # self.nfeats = nfeats
        self.pos_dim = pos_dim
        self.vel_dim = vel_dim
        self.latent_dim = latent_dim
        self.ff_size = ff_size  
        self.dropout = dropout

        self.posInEmbedding = nn.Linear(self.pos_dim, self.latent_dim)
        self.velInEmbedding = nn.Linear(self.vel_dim, self.latent_dim)
        self.sequence_pos_encoder = PositionalEncoding(self.latent_dim, self.dropout)
        self.embed_timestep = TimestepEmbedder(self.latent_dim, self.sequence_pos_encoder)

        # Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(d_model=self.latent_dim, nhead=num_heads, 
                                                    dim_feedforward=ff_size, dropout=dropout, activation=activation, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

        # Output Linear Layer
        self.posOutEmbedding = nn.Linear(self.latent_dim, self.pos_dim)
        self.velOutEmbedding = nn.Linear(self.latent_dim, self.vel_dim)

    def forward(self, x: torch.Tensor, timesteps, y=None):
        """
        x: [batch_size, max_frames, n_feats], denoted x_t in the paper
        timesteps: [batch_size] (int)
        """
        # x: [batch_size, seq_len, nfeats]
        emb = self.embed_timestep(timesteps)  # [bs, n_frames, time_embed_dim]
        # print("Emb", emb.shape)

        # Input process
        first_pos = x[:, [0], :]
        # print("First Pos", first_pos.shape)
        rest_vel = x[:, 1:, :]
        # print("Rest Vel", rest_vel.shape)

        first_pos = self.posInEmbedding(first_pos)  # [bs, 1, n_dim]
        rest_vel = self.velInEmbedding(rest_vel)  # [bs, n_frames-1, n_dim]
        x = torch.cat((first_pos, rest_vel), axis=1)  # [bs, n_frames, n_dim]
        # print("Input Embedding", x.shape)

        # Transformer Encoder
        # adding the timestep embed
        xseq = torch.cat((emb, x), axis=1)  # [bs, n_frames+1, n_dim]
        # print("Concat x and zkx", xseq.shape)

        xseq = self.sequence_pos_encoder(xseq)  # [bs, n_frames+1, n_dim]
        # print("Sequence Pos Encoder", xseq.shape)
        
        output = self.transformer_encoder(xseq)[:, 1:, :]  # , src_key_padding_mask=~maskseq)  # [bs, n_frames, n_dim]
        # print("Transformer Encoder", output.shape)

        # Output Linear
        final_pos = self.posOutEmbedding(output[:, [0], :])  # [bs, 1, pos_dim]
        final_vels = self.velOutEmbedding(output[:, 1:, :])  # [bs, n_frames-1, n_feats]
        output = torch.cat((final_pos, final_vels), axis=1)
        # print("Output Embedding", output.shape)

        return output

In [16]:
import torch.optim as optim
from torch.utils.data import DataLoader

batch_size = 1
dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, drop_last=True)

pos_dim = vel_dim = dataset[0].shape[1]

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MotionTransformer(pos_dim=pos_dim, vel_dim=vel_dim, latent_dim=64, ff_size=256, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)

In [17]:
test_sample = dataset[0].unsqueeze(0).to(device)    
res = model(test_sample, torch.tensor([0]).to(device))

res.shape

torch.Size([1, 78, 35])

In [18]:
from diffusion.diffusion import gaussian_diffusion as gd
from diffusion.diffusion.respace import SpacedDiffusion, space_timesteps

def create_gaussian_diffusion(
        diffusion_steps, # number eg 1000
        noise_schedule, # can be 'linear', 'cosine'
        sigma_small, # default True
        lambda_vel, lambda_rcxyz, lambda_fc # for geometric loss, we don't have fc, default 1 for rest
        ):
    # default params
    predict_xstart = True  # we always predict x_start (a.k.a. x0), that's our deal!
    steps = diffusion_steps
    scale_beta = 1.  # no scaling
    timestep_respacing = ''  # can be used for ddim sampling, we don't use it.
    learn_sigma = False
    rescale_timesteps = False

    betas = gd.get_named_beta_schedule(noise_schedule, steps, scale_beta)
    loss_type = gd.LossType.MSE

    if not timestep_respacing:
        timestep_respacing = [steps]

    return SpacedDiffusion(
        use_timesteps=space_timesteps(steps, timestep_respacing),
        betas=betas,
        model_mean_type=(
            gd.ModelMeanType.EPSILON if not predict_xstart else gd.ModelMeanType.START_X
        ),
        model_var_type=(
            (
                gd.ModelVarType.FIXED_LARGE
                if not sigma_small
                else gd.ModelVarType.FIXED_SMALL
            )
            if not learn_sigma
            else gd.ModelVarType.LEARNED_RANGE
        ),
        loss_type=loss_type,
        rescale_timesteps=rescale_timesteps,
        lambda_vel=lambda_vel,
        lambda_rcxyz=lambda_rcxyz,
        lambda_fc=lambda_fc,
    )

In [19]:
model = MotionTransformer(pos_dim=pos_dim, vel_dim=vel_dim, latent_dim=64, ff_size=256, num_layers=8, num_heads=4, dropout=0.1, activation="gelu").to(device)
diffusion = create_gaussian_diffusion(diffusion_steps=1000, noise_schedule="cosine", sigma_small=True, lambda_vel=1, lambda_rcxyz=1, lambda_fc=1)

In [20]:
class DefaultArgs:
    def __init__(self, save_dir, model_path, eval_model_path):
        # Base options
        self.cuda = True
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.seed = 10
        self.batch_size = 1

        # Diffusion options
        self.noise_schedule = 'cosine'
        self.diffusion_steps = 1000
        self.sigma_small = True

        # Model options
        self.arch = 'trans_enc'
        self.emb_trans_dec = False
        self.layers = 8
        self.latent_dim = 512
        self.cond_mask_prob = 0.1
        self.lambda_rcxyz = 0.0
        self.lambda_vel = 0.0
        self.lambda_fc = 0.0
        self.unconstrained = False  # This is inferred from the 'action' parameter

        # Data options
        self.dataset = 'humanml'
        self.data_dir = ""

        # Training options
        self.save_dir = save_dir
        self.overwrite = False
        self.train_platform_type = 'NoPlatform'
        self.lr = 1e-4
        self.weight_decay = 0.0
        self.lr_anneal_steps = 0
        self.eval_batch_size = 16
        self.eval_split = 'test'
        self.eval_during_training = False
        self.eval_rep_times = 3
        self.eval_num_samples = 1000
        self.log_interval = 1000
        self.save_interval = 5000
        self.num_steps = 30000
        # self.num_frames = 29
        self.resume_checkpoint = ""

        # Sampling options
        self.model_path = model_path
        self.output_dir = ''
        self.num_samples = 10
        self.num_repetitions = 3
        self.guidance_param = 2.5

        # Generate options
        self.motion_length = 6.0
        self.input_text = ''
        self.action_file = ''
        self.text_prompt = ''
        self.action_name = ''

        # Edit options
        self.edit_mode = 'in_between'
        self.text_condition = ''
        self.prefix_end = 0.25
        self.suffix_start = 0.75

        # Evaluation options
        self.eval_model_path = eval_model_path
        self.eval_mode = 'wo_mm'
        self.eval_guidance_param = 2.5


In [21]:
expname = "model-v2"

args = DefaultArgs(save_dir=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}", model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt", eval_model_path=f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/model.pt")
args.device

'cuda:0'

In [22]:
from train.training_loop import TrainLoop
TrainLoop(args, None , model, diffusion, dataloader).run_loop()

Starting epoch 0/20


  0%|          | 0/1560 [00:00<?, ?it/s]

-------------------------
| grad_norm  | 8.81e+03 |
| loss       | 3.26e+04 |
| loss_q0    | 3.2e+04  |
| loss_q1    | 3.3e+04  |
| loss_q2    | 3.26e+04 |
| loss_q3    | 3.31e+04 |
| param_norm | 54.2     |
| samples    | 1        |
| step       | 0        |
-------------------------
step[0]: loss[32630.64258]
saving model...


  0%|          | 4/1560 [00:00<02:03, 12.61it/s]

Skipping evaluation for now.


...

-------------------------
| grad_norm  | 4.06e+03 |
| loss       | 3.1e+03  |
| loss_q0    | 204      |
| loss_q1    | 238      |
| loss_q2    | 429      |
| loss_q3    | 1.16e+04 |
| param_norm | 59.5     |
| samples    | 3.1e+04  |
| step       | 3.1e+04  |
-------------------------


 87%|████████▋ | 1364/1560 [01:12<00:10, 18.76it/s]

step[31000]: loss[3104.23882]


100%|██████████| 1560/1560 [01:22<00:00, 18.87it/s]

saving model...
Skipping evaluation for now.


In [24]:
dataset[0].shape

torch.Size([78, 35])

In [25]:
num_frames = dataset[0].shape[0]
num_feats = dataset[0].shape[1]

In [26]:
sample_batch_size = 1

all_motions = []
for rep_i in range(args.num_repetitions):
    print(f'### Sampling [repetitions #{rep_i}]')

    sample_fn = diffusion.p_sample_loop

    sample = sample_fn(
        model,
        (sample_batch_size, num_frames, num_feats),
        clip_denoised=False,
        model_kwargs={"y": {}},
        skip_timesteps=0,  # 0 is the default value - i.e. don't skip any step
        init_image=None,
        progress=True,
        dump_steps=None,
        noise=None,
        const_noise=False,
    )

    all_motions.append(sample.cpu().numpy())

/home/kenji/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Sampling [repetitions #0]


100%|██████████| 1000/1000 [00:05<00:00, 182.52it/s]


### Sampling [repetitions #1]


100%|██████████| 1000/1000 [00:05<00:00, 173.28it/s]


### Sampling [repetitions #2]


100%|██████████| 1000/1000 [00:05<00:00, 180.24it/s]


In [27]:
all_motions = np.concatenate(all_motions, axis=0)
all_motions.shape

(3, 78, 35)

In [28]:
import numpy as np

def save_motions(all_motions, output_dir):
    for i, motion in enumerate(all_motions):
        filename = f"motion_{i}.npy"
        filepath = os.path.join(output_dir, filename)
        np.save(filepath, motion)
        print(f"Motion {i} saved as {filename}")

save_motions(all_motions, f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{expname}/sampled_motions/")

Motion 0 saved as motion_0.npy
Motion 1 saved as motion_1.npy
Motion 2 saved as motion_2.npy
